In [1]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import keras
from src.nimbus_analysis import autoencoder


2025-11-29 00:25:57.192571: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-29 00:25:57.223055: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-29 00:25:57.708802: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
with open('./datasets/train_X.pickle', 'rb') as file:
    train_X = pickle.load(file)
    
    
#rescale
train_X = train_X.astype('float32')/np.max(train_X)

#impute zero with median (zero exclusive)
nonzero_median = np.median(train_X[train_X > 0])
train_X[train_X == 0] = nonzero_median

train_y = train_X

#shift by 1 day between input and output
train_y = train_y[1:]
train_X = train_X[:-1]

In [ ]:


autoencoder = autoencoder.global_conv_autoencoder()

W0000 00:00:1764375959.656521   25730 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
autoencoder.fit(train_X, train_y,
                epochs=10,
                shuffle=False)

Epoch 1/10


ValueError: Dimensions must be equal, but are 180 and 12 for '{{node compile_loss/mean_squared_error/sub}} = Sub[T=DT_FLOAT](data_1, compile_loss/mean_squared_error/Squeeze)' with input shapes: [?,180,288], [?,12,20].

In [5]:
autoencoder.summary()

Model: "global_deep_conv_autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 3, 5, 4)        │         6,908 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 12, 20, 1)      │        14,733 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,641 (84.54 KB)

 Trainable params: 21,641 (84.54 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
autoencoder.save('global_autoencoder.keras')